# Explaining the Intent Classifier
This notebook demonstrates an approach to explaining why the intent classifier has classified a specific utterance in the way that it has, for the top 3 intents. It does this using a method called [LIME](https://github.com/marcotcr/lime), which runs lots of small perturbations of your utterance through the intent classifier, to create locally-faithful models which it can then explain.

You might want to use this to diagnose misclassifications, work out whether you can trust your classifier based on the words that it thinks are important, or as an educational tool.

Feedback appreciated to kalyan.dutia@ibm.com

In [ ]:
# for development
%load_ext autoreload
%autoreload 2a

In [ ]:
import sys
!{sys.executable} -m pip install lime==0.1.1.37
!{sys.executable} -m pip install ibm_watson==3.4.2

In [ ]:
import logging
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.WARNING)

import Credentials
import config
from for_csv import explainers, logger

from ibm_watson import AssistantV1

logger = logging.getLogger("explainability")

## 1. Set Topic and Authenticate Watson

In [ ]:
topic = 'puppy'

In [ ]:
# set credentials
active_adoption = Credentials.active_adoption
instance_creds = Credentials.ctx[active_adoption]

apikey = instance_creds['apikey']
#username = Credentials.ctx[active_adoption]['username']
#password = Credentials.ctx[active_adoption]['password']
url = instance_creds['url']
conversation_version = Credentials.conversation_version

print(Credentials.workspace_id[active_adoption])
workspace_id = Credentials.workspace_id[active_adoption][topic]


In [ ]:
# create Assistant instance
if 'apikey' in instance_creds:
    logger.debug("Authenticating (apikey)")
    assistant = AssistantV1(
                iam_apikey=apikey,
                version=conversation_version, 
                url=url)
    
elif 'password' in instance_creds:
    logger.debug("Authenticating (username/password)")
    assistant = AssistantV1(
                username=username,
                password=password,
                version=conversation_version,
                url=url)
    
logger.debug("Done")

## 2. Run Explainability
Fill `query_to_explain` with the utterance you want explained.

In [ ]:
# run explainability
query_to_explain = "I think my puppy is ill, how do I get my others vaccinated?"

# lime = explainers.lime_explainer(assistant, workspace_id)
lime = explainers.lime_explainer(assistant, 'Dog Advisor')
exp = lime.run(query_to_explain, print_results=False)

exp.show_in_notebook(text=True)